## Replication for "Asymmetry by Design", October 22, 2020
### By Adam Goldstein and Charlie Eaton

### Table 4: Panel Fixed Effects Estimates for Predatory Practices

Install STATA packages

In [1]:
*ssc install estout
*ssc install reghdfe

Download datasets

In [2]:
*

In [3]:
est clear
quietly use data/d_unitidasymmetry, clear
quietly xtset unitid year

foreach var in tuitionall_c_w loan_amount_borrower_c_w { 
qui eststo: reghdfe `var' multi_brand system_under all_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype* grad_rate_150_p_w, absorb(unitid year) cluster(systemid year)
qui estadd scalar schools = e(df_a)
}

quietly xtset unitid year
quietly gen l2multi_brand=l2.multi_brand
quietly replace multi_brand=l2multi_brand

quietly eststo: reghdfe ft_faculty_per_100fte_w multi_brand system_under all_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype* tuitionall_c_w, absorb(unitid year) cluster(systemid year)
qui estadd scalar schools = e(df_a)

In [4]:
quietly {
quietly use data/d_unitidasymmetry, clear
quietly xtset unitid year
  set more off  
    label var grad_rate_150_p_w "grad rate"
    foreach var in grad_rate_150_p_w  { 
qui eststo: reghdfe `var' multi_brand system_under all_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype* tuitionall_c_w, absorb(unitid year) cluster(systemid year)        
qui estadd scalar schools = e(df_a)
    }
}

In [5]:
quietly {
   quietly use data/d_opeidasymmetry, clear
keep if rankopeid==1
xtset opeid year

quietly gen l2multi_brand=l2.multi_brand
quietly replace multi_brand=l2multi_brand

quietly eststo: reghdfe mn_earn_wne_p6_c_w multi_brand  ///
all_under system_under online selective ib3.iclevel earn_pct_lo  ///
white_share_w black_share_w hisp_share_w dpcttype* tuitionall_c_w, absorb(opeid year) cluster(systemid year)
qui estadd scalar schools = e(df_a)
    
quietly replace earn_pct_lo=rpy_3yr_pct_lo

quietly eststo: reghdfe rpy_3yr_rt_supp_w multi_brand ///
all_under system_under online selective ib3.iclevel earn_pct_lo ///
white_share_w black_share_w hisp_share_w dpcttype* tuitionall_c_w, absorb(opeid year) cluster(systemid year)
qui estadd scalar schools = e(df_a)  
}

In [6]:
quietly {
    quietly use data/d_unitidasymmetry, clear

quietly collapse (rawsum) sftesale sftetotl (mean) all_under system_under grad_rate_150_p_w ///
loan_amount_borrower_c_w tuitionall_c_w selective white_share_w black_share_w hisp_share_w ///
pell_grants_per_fte_c_w ft_faculty_per_100fte_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz online ///
(first) state_n (min) iclevel (max) law_enf_frst_this_yr multi_brand, by(systemid year)

replace law_enf_frst_this_yr=1 if law_enf_frst_this_yr<. & law_enf_frst_this_yr>0
tab year if law_enf_frst_this_yr==1
xtset systemid year
quietly xtset systemid year

eststo: reghdfe f2.law_enf_frst_this_yr multi_brand system_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype*, absorb(systemid year) cluster(systemid year)

gen claimcount=.
gen mn_earn_wne_p6_c_w=.
gen rpy_3yr_rt_supp_w=.
gen earn_pct_lo=.
quietly label var pell_grants_per_fte_c_w "Pell $ / student"
quietly label var mn_earn_wne_p6_c_w "earnings"
quietly label var rpy_3yr_rt_supp_w "repay"
quietly label var claimcount "fraud claims"
quietly label var earn_pct_lo "% low-income"
label var multi_brand "multi-brand"
label var system_under "firm enrollment (1,000s)"
label var all_under "campus enrollment (1,000s)"
label var online "online"
label var selective "selective admissions"
    label drop iclevel
label define iclevel 1 "BA offered" 2 "AA offered" 
    label values iclevel iclevel
label var black_share_w "% Black"
label var hisp_share_w "% Hispanic"
label var white_share_w "% White"
label var dpcttype_arts "% degr. art"
label var dpcttype_biz "% degr. biz"
label var dpcttype_culinarycosmetic "% degr. personal service"
label var dpcttype_health "% degr. health"
label var dpcttype_law "% degr. law)"
label var dpcttype_tech "% degr. tech"
label var tuitionall_c_w "tuition (1,000s)"
label var loan_amount_borrower_c_w "loans (1,000s)"
label var ft_faculty_per_100fte_w "faculty"
label var law_enf_frst_this_yr "law enforcement"
label var grad_rate_150_p_w "undergrad grad rate"
}
qui estadd scalar schools = e(N_clust1)



. qui estadd scalar schools = e(N_clust1)


In [7]:
%html
esttab, ///
 keep(multi_brand system_under all_under online selective 1.iclevel 2.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz earn_pct_lo ///
grad_rate_150_p_w tuitionall_c_w _cons) ///
order(multi_brand system_under all_under) ///
mlabel("tuition" "borrowing" "faculty" "% grad rate" "earning" "% repay" "legal") ///
stats(r2 schools N, fmt(2 %9.0fc %9.0fc) label("r2" "schools/firms" N)) ///
se(%9.2fc) b(%9.2fc) nogaps eqlabels("")  ///
 star(* 0.05 ** 0.01 *** 0.001) html label varwidth(10)

This front-end or document format cannot display HTML

In [8]:
esttab using tables/t4_panelfixed.rtf, title(Table 4: Panel fixed effects estimates) ///
 keep(multi_brand system_under all_under online selective 1.iclevel 2.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz earn_pct_lo grad_rate_150_p_w tuitionall_c_w _cons) ///
order(multi_brand system_under all_under) ///
mlabel("tuition 1,000s" "loans (1,000s)" "faculty" "% grad rate" "earnings (1,000s)" "% repay" "legal") ///
stats(r2 schools N, fmt(2 %9.0fc %9.0fc) label("r2" "schools/firms" N)) ///
se(%9.2fc) b(%9.2fc) nogaps eqlabels("") ///
 star(* 0.05 ** 0.01 *** 0.001) label varwidth(10) replace

> 
(output written to tables/t4_panelfixed.rtf)
